In [1]:
import numpy as np 
import matplotlib.pyplot as plt
seq_dim=(6,6)

magn = np.arange(-np.prod(seq_dim), np.prod(seq_dim)+1, 2)
bins=np.linspace(magn[0]-1, magn[-1]+1, np.prod(seq_dim)+1+1)

In [2]:
magn = np.load("m_condition.npy")
pote = np.load("e_condition.npy")

In [3]:
import os
import glob
def loadmodelprediction(_dirname, epoch=None, num_batches=1, file_header="clemb"):
    dirname = _dirname
    f_logits_t = glob.glob(os.path.join(dirname, file_header+"_val_inttime*"))
    print(">>> Reading model predictions from: ", dirname)

    logits_t = np.array([np.load(f).astype(np.float16) for f in f_logits_t])
    diffusion_t = np.array([float(x.replace(os.path.join(dirname, file_header+"_val_inttime"), "").replace(".npy",""))-1 for x in f_logits_t])
    idx_order = np.argsort(diffusion_t)
    logits_t = logits_t[idx_order]
    diffusion_t = diffusion_t[idx_order]
    return logits_t, diffusion_t

In [4]:
def logits2seq(logits_t):
    seq_t = []
    for logits in logits_t:
        seq = np.argmax(logits, axis=-1)
        seq[np.where(seq==0)] = -1
        seq_t.append(seq.reshape(-1,*seq_dim))
    return seq_t

def Ising_magnetization(seq):
    data = np.sum(seq.reshape([-1,np.prod(seq_dim)]), axis=-1)
    return data



In [ ]:
condpp_t, diffusion_t = loadmodelprediction("./", file_header="rawcondpp")

In [ ]:
condxx_t, diffusion_t = loadmodelprediction("./", file_header="logits")

In [ ]:
condflow_t, diffusion_t = loadmodelprediction("./", file_header="flowprobs")

In [ ]:
print(condflow_t[-1])

In [9]:
seqc_t = logits2seq(condpp_t)
magnc_t = [Ising_magnetization(seq) for seq in seqc_t]

In [10]:
seqx_t = logits2seq(condxx_t)
magnx_t = [Ising_magnetization(seq) for seq in seqx_t]

In [11]:
seqf_t = logits2seq(condflow_t)
magnf_t = [Ising_magnetization(seq) for seq in seqf_t]

In [ ]:
memb_t, diffusion_t = loadmodelprediction("./")

In [ ]:
eemb_t, diffusion_t = loadmodelprediction("./", file_header="eemb")

In [ ]:
len(memb_t), len(eemb_t)

In [15]:
from sklearn.decomposition import PCA
emb = [np.vstack([memb_t[i], eemb_t[i]]) for i in range(len(memb_t))]

In [16]:
pc_memb_t = []
for i in range(len(memb_t)):
    pca = PCA(n_components=2)
    pca.fit(memb_t[i])
    pc_memb = pca.transform(memb_t[i])
    pc_memb_t.append(pc_memb)

In [17]:
pc_eemb_t = []
for i in range(len(eemb_t)):
    pca = PCA(n_components=2)
    pca.fit(eemb_t[i])
    pc_eemb = pca.transform(eemb_t[i])
    pc_eemb_t.append(pc_eemb)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(2*2,2))
n=-1
for i in [0,-1]:
    n += 1
    # axes[i].scatter(pc_memb_t[i][:,0], pc_memb_t[i][:,1])
    axes[i].scatter(magn, pote, c=pc_memb_t[i][:,0])


In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(2*2,2))
n=-1
for i in [0,-1]:
    n += 1
    # axes[i].scatter(pc_memb_t[i][:,0], pc_memb_t[i][:,1])
    axes[i].scatter(magn, pote, c=pc_eemb_t[i][:,0])

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(3*4,4))
n = 0
for i in [0, len(memb_t)-1]:
    # axes[i].scatter(pc_memb_t[i][:,0], pc_memb_t[i][:,1])
    cmap = axes[n].scatter(pc_memb_t[i][:,0], pc_eemb_t[i][:,0], c=magnc_t[i])
    cbar = plt.colorbar(cmap, ax=axes[n])
    n += 1

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=1, figsize=(3*5,2))
n = 0
# for i in [0, len(memb_t)-1]:
for i in np.linspace(0, len(memb_t)-1, 5).astype(int):
    # axes[i].scatter(pc_memb_t[i][:,0], pc_memb_t[i][:,1])
    cmap = axes[n].scatter(magn-36, pote, c=magnc_t[i], s=1)
    cbar = plt.colorbar(cmap, ax=axes[n])
    n += 1

In [ ]:
plt.figure(figsize=(5,5))
_ = plt.hist2d(magn-36, magnc_t[-1], bins=72)
plt.xlabel("m condition")
plt.ylabel("Generated m")

In [ ]:
plt.figure(figsize=(9,4))
plt.subplot(121)
_ = plt.hist(magn-36, bins=72)
plt.xlabel("m condition")
plt.subplot(122)
_ = plt.hist(magnc_t[-1]//(seq_dim[0]/6)**2, bins=72)
plt.xlabel("Generated m")

In [ ]:
_ = plt.hist(magnx_t[-1], bins=72)

In [ ]:
_ = plt.hist(magnc_t[-1], bins=72)

In [ ]:
_ = plt.hist(magnf_t[-1], bins=72)

In [26]:
import torch
def RC(logits):
    assert len(logits.shape) == 4
    assert logits.shape[-1] == 2
    B = logits.shape[0]
    RC = torch.sum(logits*torch.tensor([-1,1], device=logits.device)[None,None,None,:], dim=-1)
    RC = torch.sum(RC.reshape(B, -1), dim=-1)
    return RC.reshape(-1,1)

In [27]:
cls = np.load("cls.npy")
e = np.load("energy_op_inp.npy")
pp = np.load("rawcondpp.npy")
seq = np.argmax(pp, axis=-1)
seq[np.where(seq==0)] = -1
magn_pp = np.sum(seq.reshape([-1,np.prod(seq_dim)]), axis=-1)
# xt = np.load("xt.npy")
# seq = np.argmax(xt, axis=-1)
# seq[np.where(seq==0)] = -1
# magn_xt = np.sum(seq.reshape([-1,np.prod(seq_dim)]), axis=-1)
CEloss = np.load("CELoss.npy")
RCloss = np.load("RCLoss.npy")
MSEloss = np.load("MSEloss.npy")

In [28]:
rc_pp = RC(torch.from_numpy(pp)).numpy()

In [29]:
rc_logits = np.load("rc_logits.npy")

In [30]:
kderclogits = np.load("kde_rc_logits.npy")
kdercseq = np.load("kde_rc_seq.npy")

In [ ]:
_ = plt.hist(rc_pp, bins=100)

In [ ]:
_ = plt.hist(rc_logits, bins=100)

In [ ]:
# plt.plot(np.arange(37)*2, RCloss)
plt.plot(np.arange(np.prod(seq_dim)+1)*2-np.prod(seq_dim), kderclogits)
plt.plot(np.arange(np.prod(seq_dim)+1)*2-np.prod(seq_dim), kdercseq)


In [ ]:
plt.plot(np.arange(np.prod(seq_dim)+1)*2-np.prod(seq_dim), RCloss)

In [ ]:
plt.scatter(cls,e,c=MSEloss)

In [ ]:
plt.scatter(cls,e,c=CEloss.mean(-1))